In [1]:
import sys
sys.path.insert(0, "..")

from src.db_ingestion.chroma_client import query_to_collection
from src.talent_selection_flow.crews.cv_to_job_crew.crew import CVToJobCrew

In [19]:
from src.talent_selection_flow.crews.cv_to_job_crew.schemas import GapAnalysisOutput, JobGapAnalysis

In [31]:
from src.config.paths import REPORT_OUTPUT_PATH, JOBS_PATH_PROCESSED
str(REPORT_OUTPUT_PATH)

'C:\\Users\\tania\\Desktop\\multiagent system arnau\\talent_selection_flow_crewai\\data\\reports\\latest_report.md'

In [32]:
JOBS_PATH_PROCESSED

WindowsPath('C:/Users/tania/Desktop/multiagent system arnau/talent_selection_flow_crewai/data/processed/jobs_processed.csv')

In [10]:
cv_example = "Experienced Software Engineer with 5+ years in Python, specialized in building scalable AI agents and distributed systems using CrewAI and FastAPI. From Houston"
top_k = 3

In [14]:
from src.db_ingestion.metadata_extraction_crew.crews import CVMetadataExtractorCrew, JobMetadataExtractorCrew
from src.db_ingestion.metadata_extraction_crew.enums import ExperienceLevel, EducationLevel, EmploymentType
import json

# TODO add metadata extraction crew
metadata = CVMetadataExtractorCrew(
    guardrail_max_retries=3,
    verbose=False,
    human_input=True,
).crew().kickoff(inputs={
    "content": cv_example,
    "educationlevel_options": "/".join(EducationLevel),
    "experiencelevel_options": "/".join(ExperienceLevel),
})

╭────────────────────────────────────────── 💬 Human Feedback Required ───────────────────────────────────────────╮
│                                                                                                                 │
│  Provide feedback on the Final Result above.                                                                    │
│                                                                                                                 │
│  • If you are happy with the result, simply hit Enter without typing anything.                                  │
│  • Otherwise, provide specific improvement requests.                                                            │
│  • You can provide multiple rounds of feedback until satisfied.                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

2026-02-16 10:44:41.534 | DEBUG    | src.db_ingestion.metadata_extraction_crew.guardrails:validate_cvmetadata_schema:12 - Guardrail input:
{"skills": "Python, CrewAI, FastAPI", "industries": "AI, Software Development", "experience_level": "senior", "country": "US", "summary": "Experienced Software Engineer with 5+ years in Python, specialized in building scalable AI agents and distributed systems using CrewAI and FastAPI.", "education_level": "unknown", "languages": "unknown"}


In [15]:
metadata_dict = json.loads(metadata.raw)

# Get matches from jobs collection
related_jobs = query_to_collection(
    collection_name="jobs",
    query_text=cv_example,
    country=metadata_dict.get("country"),
    top_k=3,
)

2026-02-16 10:44:46.559 | INFO     | src.db_ingestion.chroma_client:query_to_collection:152 - Initiating vector search in collection 'jobs' (Top K: 3)
2026-02-16 10:44:46.970 | DEBUG    | src.db_ingestion.chroma_client:query_to_collection:174 - Final formatted results:
{'2417': {'title': 'Sr. Developer - Python/Java', 'similarity': 0.8087, 'skills': 'Python, Oracle, Teradata, Linux, Java, shell scripting, Integration, XML, XLST, WCF, SOAP, JSON, RESTful services, systems, application frameworks, database optimization, software development projects, product development, banking, trading, payment, financial applications, security, single sign on, device fingerprinting, multi factor authentication, adaptive authentication, SDLC methodologies, Agile, Rational Unified Process, deployment automation, code reviews, test driven development', 'industries': 'Banking', 'experience_level': 'senior', 'summary': 'Leading bank is seeking a Senior Software Engineer (Python/ Java) in San Ramon for a co

In [16]:
related_jobs

{'2417': {'title': 'Sr. Developer - Python/Java',
  'similarity': 0.8087,
  'skills': 'Python, Oracle, Teradata, Linux, Java, shell scripting, Integration, XML, XLST, WCF, SOAP, JSON, RESTful services, systems, application frameworks, database optimization, software development projects, product development, banking, trading, payment, financial applications, security, single sign on, device fingerprinting, multi factor authentication, adaptive authentication, SDLC methodologies, Agile, Rational Unified Process, deployment automation, code reviews, test driven development',
  'industries': 'Banking',
  'experience_level': 'senior',
  'summary': 'Leading bank is seeking a Senior Software Engineer (Python/ Java) in San Ramon for a contract role. The client is seeking a senior software engineer contractor for the Shared Services team.',
  'country': 'US'},
 '16162': {'title': 'Senior Quality Assurance Engineer (remote/telecommute)',
  'similarity': 0.8085,
  'skills': 'manual and automated